In [1]:
# import python built-in libraries
import os.path

# import external libraries
import netCDF4 
import pandas as pd
import numpy as np
import yaml
from datetime import datetime, timedelta
from cftime import num2date, date2num

# import user libraries
from utils import description_module
from utils import conversion_module

# load configuration file
with open('config.yml') as f:
    cfg = yaml.load(f, Loader = yaml.FullLoader)

In [2]:

dataPath = "./data/"
# dataPath = "./raw_data/Maebong_and_Moonam/"
 
files = os.listdir(dataPath)

datasetDict = {}
for idx,nc_type_file in enumerate(files):
    if nc_type_file.split(".")[-1] == 'nc':
        datasetDict[nc_type_file] = netCDF4.Dataset(dataPath+nc_type_file,mode="r")

In [3]:
ncfile_name = '20210731_test_example_2.nc'
output_dataset = datasetDict[ncfile_name]

# 하단 데이터를 갖고 있습니다.
output_dataset.variables.keys()

#mdww : mean direction of wind waves
#mpww : mean period of wind waves
#mwd : mean wave direction
#mwp : mean wave period
#wmb : Model bathymetry
#dwi : Ocean surface stress equivalent 10m neutral wind direction
#swh : Significant height of combined wind waves and swell
#shww : significant height of wind waves

dict_keys(['longitude', 'latitude', 'time', 'u100', 'v100'])

In [ ]:
info_dict, meta_dict = conversion_module.netCDF4_to_CSV(output_dataset)
data_df = conversion_module.dict_to_dataframe(info_dict, meta_dict)
data_df = conversion_module.measure_wind_info(data_df)

fname = str(meta_dict['latitude'][0])+"(lat) "+str(meta_dict['longitude'][0])+"(lon) located data"+'.csv'
conversion_module.save_df(data_df, cfg['project_title'], fname)

del info_dict
del meta_dict

In [5]:
data_df

,latitude,longitude,time,u100,v100,100m_wind_direction,100m_wind_speed
0,34.24,127.09,2018-01-01 00:00:00,1.1992001371319785,-3.387745165685549,340.50696351005513 degree,3.5937304123324503 meter / second
1,34.24,127.09,2018-01-01 01:00:00,1.5731244834276006,-3.3271057486084805,334.69424608267923 degree,3.6802653848306854 meter / second
2,34.24,127.09,2018-01-01 02:00:00,1.1999525000017885,-4.063085522995914,343.5465295913808 degree,4.23657290382681 meter / second
3,34.24,127.09,2018-01-01 03:00:00,1.0532417403888177,-4.451842333846161,346.6893861979315 degree,4.574736968299062 meter / second
4,34.24,127.09,2018-01-01 04:00:00,1.3669770570996316,-4.10877275504028,341.59786209610036 degree,4.330200899172931 meter / second
...,...,...,...,...,...,...,...
26299,34.24,127.09,2020-12-31 19:00:00,2.157710433270052,-5.925463236513548,339.9913068494928 degree,6.306094582316066 meter / second
26300,34.24,127.09,2020-12-31 20:00:00,1.8891168887478451,-4.996766410593515,339.2900048859797 degree,5.34195069065486 meter / second
26301,34.24,127.09,2020-12-31 21:00:00,1.5309921627182348,-3.8313266549890357,338.2184273715124 degree,4.125893956409221 meter / second
26302,34.24,127.09,2020-12-31 22:00:00,1.78378608697443,-3.0945162036553415,330.03936529673 degree,3.5718235313028854 meter / second
